<a href="https://colab.research.google.com/github/Kobayashi139/food-supply/blob/main/%E5%8A%A0%E9%87%8D%E5%92%8C%E6%B3%95%E3%83%91%E3%83%AC%E3%83%BC%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ライブラリのインポート

In [1]:
# =================================================================
# モジュール1: 環境設定とライブラリのインポート
# =================================================================
!pip install -q pyomo plotly
!apt-get install -y -qq glpk-utils

import pyomo.environ as pyo
import pandas as pd
import numpy as np
import random, time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

### 需要乱数の生成



In [2]:
try:
    # 乱数シードを固定し、毎回同じ乱数が発生するようにする
    np.random.seed(42)
    solver = pyo.SolverFactory('glpk')
except pyo.common.errors.ApplicationError:
    print("GLPK solver not found. Please ensure it is installed and in your system's PATH.")
    solver = None

### パラメータとシナリオ

In [21]:
# =================================================================
# モジュール2: パラメータとシナリオの定義
# =================================================================

# --- 製品タイプの集合 ---
# I = {f: 生鮮品（檸檬）, a: 加工品A（檸檬タルト）, b: 加工品B（檸檬シロップ）}
products = ['f', 'a', 'b']
processed_products = ['a', 'b']

# --- 単位統一のための基本データ設定 ---
# 製品1個あたりの基本情報
# タルト('a')
WEIGHT_TART_g = 350      # 容量 (g/個)
LEMON_PER_TART_ml = 50    # 檸檬含量 (ml/個)
LIFESPAN_TART_days = 120   # 保存期間 (日)
SALES_TART = 3800 # 値段 (円/個)
# シロップ('b')
WEIGHT_SYRUP_g = 400     # 容量 (g/個)
LEMON_PER_SYRUP_ml = 200  # 檸檬含量 (ml/個)
LIFESPAN_SYRUP_days = 365 # 保存期間 (日)
SALES_SYRUP = 2200 # 値段 (円/個)
# 生鮮品('f') - 檸檬
LIFESPAN_FRESH_days = 60  # 保存期間 (日、仮定)
SALES_FRESH = 500 # 値段 (円/kg)

# 1個あたりの重量 (kg/個)
WEIGHT_PER_PIECE = {
    'a': WEIGHT_TART_g / 1000,
    'b': WEIGHT_SYRUP_g / 1000
}
# 製品1個の製造に必要な檸檬の重量 (kg/個) - 檸檬の密度を約1g/mlと仮定
LEMON_KG_PER_PIECE = {
    'a': LEMON_PER_TART_ml / 1000,
    'b': LEMON_PER_SYRUP_ml / 1000
}
print(WEIGHT_PER_PIECE)
print(LEMON_KG_PER_PIECE)

# 電気代・ガス代 (円/月)
COST_FREEZER_monthly = 389.17 # 冷凍庫
COST_OVEN_monthly = 155       # オーブン
COST_GAS_monthly = 1048.5     # ガスコンロ

# 梱包費など (円/個)
PACKING_COST_PER_PIECE = {
    'a': 63,
    'b': 230
}

# --- コスト、劣化率、効率などのパラメータ ---

# 1. 保管コスト h (円/kg・月)
# 月額コストを、その保管場所で保管可能な最大重量(kg)で割って算出
h_a = COST_FREEZER_monthly / 80  # 冷凍庫100Lに80kg保管可能と仮定
h_b = 1                          # 常温保管コストは微小と仮定
h_f = 5                          # 冷蔵庫の月額コスト

# 2. 劣化率 gamma (%/月)
# 1ヶ月(30日)で劣化する割合。 (30日 / 保存期間) で計算、上限1.0(100%)
gamma_f = min(1.0, 30 / LIFESPAN_FRESH_days)
gamma_a = min(1.0, 30 / LIFESPAN_TART_days)
gamma_b = min(1.0, 30 / LIFESPAN_SYRUP_days)


# 3. 加工効率 e (kg-output / kg-input)
e_a = 0.8
e_b = 0.9

# 4. 加工コスト p (円/kg-input)
# 原料(檸檬)1kgを加工するためのコスト」
# (設備費＋梱包費)を、対応する「原料投入量(kg-input)」で割る

# タルト('a'): 月産60個と仮定
p_a_monthly_cost = COST_OVEN_monthly + (PACKING_COST_PER_PIECE['a'] * 80)
p_a_monthly_weight_kg = WEIGHT_PER_PIECE['a'] * 60 # 月間「生産量(output)」
#「投入量(input)」を効率e_aで逆算
p_a_monthly_input_kg = p_a_monthly_weight_kg / e_a
# コストを「投入量」で割る
p_a = p_a_monthly_cost / p_a_monthly_input_kg

# シロップ('b'): 月産30個と仮定
p_b_monthly_cost = COST_GAS_monthly + (PACKING_COST_PER_PIECE['b'] * 40)
p_b_monthly_weight_kg = WEIGHT_PER_PIECE['b'] * 30 # 月間「生産量(output)」
# 対応する「投入量(input)」を効率e_bで逆算
p_b_monthly_input_kg = p_b_monthly_weight_kg / e_b
# コストを「投入量」で割る
p_b = p_b_monthly_cost / p_b_monthly_input_kg


# h_i: 保管コスト, p_i: 加工コスト, γ_i: 劣化率, φ_i: 廃棄コスト, e_i: 加工効率, o: 1単位あたりの機会損失コスト
params = {
    'h': {'f': h_f, 'a': h_a, 'b': h_b},
    'p': {'f': 0, 'a': p_a, 'b': p_b},
    'gamma': {'f': gamma_f, 'a': gamma_a, 'b': gamma_b},
    'e': {'f': 1, 'a': e_a, 'b': e_b},
    'phi': {'f': 1000, 'a': 1000, 'b': 1500}, # 処理コスト重視(円/kg)
    'o': {'f': 500, 'a': 2000, 'b': 1500}   # 粗利寄りの計上(円/kg)
}

print(params['p'])

# --- 能力に関するパラメータ (単位: kg) ---
capacities = {
    # U: 在庫上限 (kg)
    'U': {'f': 40, 'a': 80, 'b': 120},
    # u: 月間の最大生産量 (kg/月)
    'u': {
        'a': 60 * WEIGHT_PER_PIECE['a'], # 月最大60個生産と仮定
        'b': 30 * WEIGHT_PER_PIECE['b']   # 月最大30個生産と仮定
    }
}

# --- 供給と初期在庫 (単位: kg) ---
C_supply = 25 # 月間の生鮮品(檸檬)の供給量 (kg)
initial_inventory_post = {'f': 5, 'a': 0, 'b': 0} # 加工後在庫 (kg)
initial_inventory_pre = {'a': 0, 'b': 0}          # 加工前在庫(原料檸檬) (kg)

# --- 需要パラメータ ---
demand_params = {
    'mean': {
        'f': 5, # 生の檸檬の月間需要 (kg)
        'a': 30 * WEIGHT_PER_PIECE['a'], # タルト: 月需要(個
        'b': 20 * WEIGHT_PER_PIECE['b']  # シロップ: 月需要(個
    },
    'std': {
        'f': 2,
        'a': 15 * WEIGHT_PER_PIECE['a'],
        'b': 10 * WEIGHT_PER_PIECE['b']
    }
}

num_scenarios_per_period = 20 # 生成するシナリオの数

params['waste_limit'] = 0.5 * C_supply

# --- 環境負荷係数（ポイント/kg = おおむね kgCO2e/kg に対応） ---

waste_emission_factors = {
    'low': {
        'f': 0.90,
        'a': 0.90,
        'b': 0.90
    },
      'middle': {
        # [出典] 農業LCI(3.64t-CO2/百万円) (1.09)
        'f': 2.00,

        # 原料(1.09) + 電気加工(0.26) + 複合焼却(0.16) = 1.51
        'a': 2.51,

        # 原料(1.09) + ガス加工(0.12) + 瓶ロス/処理(0.33) = 1.54
        'b': 3.54
    },
    'high': {
        'f': 4.95,
        'a': 5.65,
        'b': 5.50
    }
}

process_emission_factors = {
    'low': {
        'a': 0.05,
        'b': 0.05
    },
    'middle': {
        # [出典] 中国電力R5 (0.511 kg/kWh) * 0.5 kWh/kg
        'a': 2.26,
        # [出典] 広島ガスR5 (2.05 kg/m3) * 0.06 m3/kg
        'b': 2.12
    },
    'high': {
        'a': 0.50,
        'b': 0.30
    }
}


{'a': 0.35, 'b': 0.4}
{'a': 0.05, 'b': 0.2}
{'f': 0, 'a': 197.9047619047619, 'b': 768.6375}


### 環境負荷没

In [5]:
# # --- 環境負荷係数 (単位: ポイント/kg) ---
# # 廃棄物1kgあたり、または加工プロセスで原料1kg使用あたりの負荷
# waste_emission_factors = {
#     'low': {'f': 1.0, 'a': 1.5, 'b': 2.0},
#     'middle':{'f': 4.0, 'a': 4.5, 'b': 5.0},
#     'high': {'f': 8.0, 'a': 8.5, 'b': 9.0}
# }
# process_emission_factors = {
#     'low': {'a': 1.8, 'b': 2.5},
#     'middle':{'a': 4.5, 'b': 3.0},
#     'high': {'a': 6.0, 'b': 4.5}
# }


# # 廃棄による排出量（廃棄 1 kg 当たりの kgCO2e）
# waste_emission_factors = {
#     'low':    {'f': 0.30, 'a': 1.50, 'b': 0.50},   # 軽め（輸送短・低入力）
#     'middle': {'f': 0.60, 'a': 3.00, 'b': 1.20},   # 中間（代表値）
#     'high':   {'f': 1.20, 'a': 5.00, 'b': 2.00}    # 高め（長距離輸送・高付加価値加工品）
# }

# # 加工による排出量（加工 1 kg 当たりの kgCO2e）
# # ここは「加工投入量 1 kg が引き起こすエネルギー由来排出」を想定
# process_emission_factors = {
#     'low':    {'a': 1.0,  'b': 0.2},
#     'middle': {'a': 2.5,  'b': 0.6},
#     'high':   {'a': 4.0,  'b': 1.2}
# }

# # 廃棄による排出量
# # waste_emission_factors = {
# #     'low':    {'f': 500.0, 'a': 500.0, 'b': 400.0},
# #    "middle":  { 'f': 600,  'a': 700, 'b': 1200},
# #     'high':   {'f': 2000.0, 'a': 2000.0, 'b': 1500.0}
# # }
# waste_emission_factors = {
#     'low':    {'f': 0.4, 'a': 0.5, 'b': 0.9},
#     'middle': {'f': 0.6, 'a': 0.7, 'b': 1.2},
#     'high':   {'f': 0.9, 'a': 1.0, 'b': 1.6}
# }

# # 加工による排出量
# # process_emission_factors = {
# #     'low':    {'a': 500.0, 'b': 300.0},
# #     'middle': {'a': 2000.0, 'b': 100.0},
# #     'high':   {'a': 1500.0, 'b': 1000.0}
# # }
# process_emission_factors = {
#     'low':    {'a': 0.2, 'b': 0.1},
#     'middle': {'a': 0.5, 'b': 0.3},
#     'high':   {'a': 1.5, 'b': 1.0}
# }

# ==========================================
# 現実的なCO2排出係数 (単位: kg-CO2e / kg-廃棄物)
# ==========================================
# 廃棄による排出量
# f (生鮮): 水分多め、運搬と助燃剤のみ。約 0.15
# a (タルト): 複合素材(プラ・紙)を含む焼却。約 0.50
# b (シロップ): ガラス瓶。重量があり運搬負荷大＋製造エネルギーの損失。約 1.20
# waste_emission_factors = {
#     'low':    {'f': 0.10, 'a': 0.30, 'b': 0.80},
#     'middle': {'f': 0.15, 'a': 0.50, 'b': 1.20},  # これを採用
#     'high':   {'f': 0.20, 'a': 0.80, 'b': 1.80}
# }

# # 加工による排出量 (単位: kg-CO2e / kg-製品)
# # 加工プロセスで使う電気・ガス由来のCO2
# # a (タルト): オーブン焼成はエネルギーを多く使う。約 0.8
# # b (シロップ): 煮込み(ガス)は比較的効率が良いが時間はかかる。約 0.4
# process_emission_factors = {
#     'low':    {'a': 0.5, 'b': 0.2},
#     'middle': {'a': 0.8, 'b': 0.4}, # これを採用
#     'high':   {'a': 1.2, 'b': 0.6}
# }

In [4]:
# # 現実的に考える
# food = 1.76
# paper_scraps = 0.1082
# glass_waste = 0.0117


## パレートフロンティア

In [22]:
# =================================================================
# モジュール3: 多目的最適化モデルを解く関数 (目的関数変更版)
# =================================================================
def solve_multi_objective_model(initial_inv_post, initial_inv_pre, supply, scenarios, params, caps, env_weight, normalization_limits=None):
    model = pyo.ConcreteModel("Multi_Objective_Model")

    # 集合
    model.I = pyo.Set(initialize=products)
    model.I_processed = pyo.Set(initialize=processed_products)
    model.S = pyo.Set(initialize=scenarios.keys())

    # 決定変数
    model.X = pyo.Var(model.I, domain=pyo.NonNegativeReals) # 原料配分量
    model.P_actual = pyo.Var(model.I_processed, domain=pyo.NonNegativeReals) # 加工投入量
    model.I_end_post = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals) # 期末在庫(加工後)
    model.I_end_pre = pyo.Var(model.S, model.I_processed, domain=pyo.NonNegativeReals) # 期末在庫(加工前)
    model.S_sold = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals) # 販売量
    model.W_waste_post = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals) # 廃棄量(加工後)
    model.W_waste_pre = pyo.Var(model.S, model.I_processed, domain=pyo.NonNegativeReals) # 廃棄量(加工前)
    model.UnmetDemand = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals) # 未充足需要

    # --- 目的関数要素の定義 ---
    economic_cost_expr = (
        # 保管コスト
        sum(scenarios[s]['prob'] * sum(params['h'][i] * model.I_end_post[s, i] for i in model.I) for s in model.S)
        # 加工前保管コスト
        + sum(scenarios[s]['prob'] * sum(params['h']['f'] * model.I_end_pre[s, i] for i in model.I_processed) for s in model.S)
        # 加工コスト
        + sum(params['p'][i] * model.P_actual[i] for i in model.I_processed)
        # 廃棄コスト (加工後)
        + sum(scenarios[s]['prob'] * sum(params['phi'][i] * model.W_waste_post[s, i] for i in model.I) for s in model.S)
        # 廃棄コスト (加工前)
        + sum(scenarios[s]['prob'] * sum(params['phi']['f'] * model.W_waste_pre[s, i] for i in model.I_processed) for s in model.S)
        # 機会損失コスト
        + sum(scenarios[s]['prob'] * sum(params['o'][i] * model.UnmetDemand[s, i] for i in model.I) for s in model.S)
    )
    environmental_score_expr = (
        # 廃棄による環境負荷
        sum(scenarios[s]['prob'] * sum(waste_emission_factors['middle'][p] * model.W_waste_post[s, p] for p in model.I) for s in model.S)
        + sum(scenarios[s]['prob'] * sum(waste_emission_factors['middle']['f'] * model.W_waste_pre[s, i] for i in model.I_processed) for s in model.S)
        # 加工による環境負荷
        + sum(process_emission_factors['middle'][p] * model.P_actual[p] for p in model.I_processed)
    )

    # 経済コスト
    model.economic_cost = pyo.Expression(expr=economic_cost_expr)
    # 環境負荷スコア
    model.environmental_score = pyo.Expression(expr=environmental_score_expr)

    # # ---------------------------------------------------------
    # # 目的関数: (1 - λ) * コスト + λ * 環境負荷
    # # env_weight は 0.0 ~ 1.0 の範囲を想定
    # # ---------------------------------------------------------
    # model.total_objective = pyo.Objective(
    #     expr=(1.0 - env_weight) * model.economic_cost + env_weight * model.environmental_score,
    #     sense=pyo.minimize
    # )
    # --- 目的関数の定義 (正規化) ---
    def objective_rule(model):
        cost_term = model.economic_cost
        env_term = model.environmental_score

        if normalization_limits:
            c_min, c_max, e_min, e_max = normalization_limits
            # ゼロ除算を防ぐため分母に微小値(1e-5)を加算
            norm_cost = (cost_term - c_min) / (c_max - c_min + 1e-5)
            norm_env  = (env_term - e_min) / (e_max - e_min + 1e-5)
            return (1.0 - env_weight) * norm_cost + env_weight * norm_env
        else:
            # 正規化なし（従来の計算）
            return (1.0 - env_weight) * cost_term + env_weight * env_term

    model.total_objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)


    # 制約
    model.constraints = pyo.ConstraintList()

    # 総供給量制約
    model.constraints.add(sum(model.X[i] for i in model.I) == supply)

    # 加工能力と原料利用の制約
    for i in model.I_processed:
        # 月間最大生産量
        model.constraints.add(model.P_actual[i] * params['e'][i] <= caps['u'][i])
        # 加工投入量は、(繰越在庫＋新規割当)を超えられない
        model.constraints.add(model.P_actual[i] <= initial_inv_pre.get(i, 0) * (1 - params['gamma']['f']) + model.X[i])

    # 各シナリオにおけるバランス制約
    for s in model.S:
        # 在庫バランス (加工後) - 生鮮品
        model.constraints.add(initial_inv_post['f'] * (1 - params['gamma']['f']) + model.X['f'] == model.S_sold[s, 'f'] + model.I_end_post[s, 'f'])

        for i in model.I_processed:
            # 在庫バランス (加工後) - 加工品
            model.constraints.add(initial_inv_post.get(i, 0) * (1 - params['gamma'][i]) + model.P_actual[i] * params['e'][i] == model.S_sold[s, i] + model.I_end_post[s, i])

            # 在庫バランス (加工前)
            model.constraints.add(
                (initial_inv_pre.get(i, 0) * (1 - params['gamma']['f']) + model.X[i]) - model.P_actual[i] == model.I_end_pre[s, i]
            )
            # 廃棄量計算 (加工前)
            model.constraints.add(model.W_waste_pre[s, i] == initial_inv_pre.get(i, 0) * params['gamma']['f'])

        for i in model.I:
            # 販売量は需要以下
            model.constraints.add(model.S_sold[s, i] <= scenarios[s]['d'][i])
            # 未充足需要の計算
            model.constraints.add(model.S_sold[s, i] + model.UnmetDemand[s, i] == scenarios[s]['d'][i])
            # 廃棄量計算 (加工後)
            model.constraints.add(model.W_waste_post[s, i] == initial_inv_post.get(i, 0) * params['gamma'][i])
            # 在庫上限
            model.constraints.add(model.I_end_post[s, i] <= caps['U'][i])

    if solver is None:
        return None, None

    results = solver.solve(model, tee=False)
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        decisions = {
            'X': {i: pyo.value(model.X[i]) for i in model.I},
            'P_actual': {i: pyo.value(model.P_actual[i]) for i in model.I_processed}
        }
        return decisions, model
    else:
        return None, None

def calculate_normalization_bounds(initial_inv_post, initial_inv_pre, supply, scenarios, params, caps):
    print("正規化のための基準値(レンジ)を計算中...")

    # 1. コスト最優先 (lambda=0) で解く -> C_min と E_max (仮) を取得
    decisions_c, model_c = solve_multi_objective_model(initial_inv_post, initial_inv_pre, supply, scenarios, params, caps, env_weight=0.0, normalization_limits=None)
    c_min = pyo.value(model_c.economic_cost)
    e_at_c_min = pyo.value(model_c.environmental_score)

    # 2. 環境最優先 (lambda=1) で解く -> E_min と C_max (仮) を取得
    decisions_e, model_e = solve_multi_objective_model(initial_inv_post, initial_inv_pre, supply, scenarios, params, caps, env_weight=1.0, normalization_limits=None)
    e_min = pyo.value(model_e.environmental_score)
    c_at_e_min = pyo.value(model_e.economic_cost)

    # 範囲を定義
    # 理論的な最大値は相互のケースから取るのが一般的 (Payoff Table)
    c_max = max(c_at_e_min, c_min * 1.5) # 安全のため少し余裕を持たせる場合もあるが、ここでは値をそのまま採用
    e_max = max(e_at_c_min, e_min * 1.5)

    print(f"  Cost Range: {c_min:,.0f} ~ {c_max:,.0f}")
    print(f"  Env  Range: {e_min:,.1f} ~ {e_max:,.1f}")

    return c_min, c_max, e_min, e_max

# =================================================================
# モジュール4: シミュレーション
# =================================================================
def run_two_stage_simulation(simulation_periods, env_weight, initial_inventory_post, initial_inventory_pre, master_scenarios, normalization_limits=None, verbose=True):
    if verbose:
        print("\n" + "="*80)
        print(f"詳細シミュレーションを開始します (環境重視度 λ = {env_weight:.2f})")
        print("="*80)

    # (t=0 の初期在庫)
    inv_post = initial_inventory_post.copy()
    inv_pre = initial_inventory_pre.copy()

    full_history = []
    for t in range(1, simulation_periods + 1):
        # シナリオの生成
        scenarios = master_scenarios[t-1]
        # (t期の) 期首在庫(inv_post, inv_pre) をモジュール3に渡す
        decisions, solved_model = solve_multi_objective_model(inv_post, inv_pre, C_supply, scenarios, params, capacities, env_weight, normalization_limits)

        if not decisions:
            print(f"❌ 期 {t} で最適解なし。中断。")
            break

        # (t期の) モジュール3が計算した「期末在庫」を取得 -> 結果を記録
        period_data = {
            'period': t,
            'cost': pyo.value(solved_model.economic_cost),
            'env_score': pyo.value(solved_model.environmental_score),
            'weight': env_weight,
            # 意思決定変数
            'X_f': decisions['X']['f'], 'X_a': decisions['X']['a'], 'X_b': decisions['X']['b'],
            'P_a': decisions['P_actual']['a'], 'P_b': decisions['P_actual']['b'],
            # 在庫量 (期待値)
            'Inv_post_f': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_post[s, 'f']) for s in scenarios),
            'Inv_post_a': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_post[s, 'a']) for s in scenarios),
            'Inv_post_b': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_post[s, 'b']) for s in scenarios),
            'Inv_pre_a': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_pre[s, 'a']) for s in scenarios),
            'Inv_pre_b': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_pre[s, 'b']) for s in scenarios),
            # 廃棄量 (期待値)
            'W_post_f': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_post[s, 'f']) for s in scenarios),
            'W_post_a': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_post[s, 'a']) for s in scenarios),
            'W_post_b': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_post[s, 'b']) for s in scenarios),
            'W_pre_a': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_pre[s, 'a']) for s in scenarios),
            'W_pre_b': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_pre[s, 'b']) for s in scenarios),
        }
        full_history.append(period_data)

        # (t+1期の) 「期首在庫」として変数を更新
        inv_post = {p: period_data[f'Inv_post_{p}'] for p in products}
        inv_pre = {p: period_data[f'Inv_pre_{p}'] for p in processed_products}

    return pd.DataFrame(full_history)

# =================================================================
# モジュール5: パレートフロンティア分析
# =================================================================
def run_pareto_analysis(env_weight_list, simulation_periods, master_scenarios, normalization_limits=None, title="パレートフロンティア分析"):
    print("="*80 + f"\n{title}を開始...\n" + "="*80)
    results_list = []
    start_time = time.time()

    for weight in env_weight_list:
        df_history = run_two_stage_simulation(simulation_periods, env_weight=weight,
                                                  initial_inventory_post=initial_inventory_post,
                                                  initial_inventory_pre=initial_inventory_pre,
                                                  master_scenarios=master_scenarios,
                                                  normalization_limits=normalization_limits,
                                                  verbose=False)

        if not df_history.empty:
            total_cost = df_history['cost'].sum()
            total_env_score = df_history['env_score'].sum()
            total_objective = (1 - weight) * total_cost + weight * total_env_score

            # 結果表示用の目的関数値（正規化前の生の値での加重和、または正規化後か。ここでは比較のため単純な加重和）
            # ※注意: 内部最適化では正規化されているが、ここで表示するのは「結果としてのコストと環境負荷」
            results_list.append({
                'weight': weight,
                'total_cost': total_cost,
                'total_env_score': total_env_score,
                'total_objective': total_objective
            })



    print(f"\n完了: {time.time() - start_time:.2f} 秒")
    return pd.DataFrame(results_list)

# =================================================================
# モジュール6: 1期間の詳細分析と結果表示
# =================================================================
def analyze_and_print_period_details(model, scenarios, params):
    holding_cost_post = sum(scenarios[s]['prob'] * sum(params['h'][i] * pyo.value(model.I_end_post[s, i]) for i in model.I) for s in model.S)
    holding_cost_pre = sum(scenarios[s]['prob'] * sum(params['h']['f'] * pyo.value(model.I_end_pre[s, i]) for i in model.I_processed) for s in model.S)
    total_holding_cost = holding_cost_post + holding_cost_pre
    processing_cost = sum(params['p'][i] * pyo.value(model.P_actual[i]) for i in model.I_processed)
    waste_cost_post = sum(scenarios[s]['prob'] * sum(params['phi'][i] * pyo.value(model.W_waste_post[s, i]) for i in model.I) for s in model.S)
    waste_cost_pre = sum(scenarios[s]['prob'] * sum(params['phi']['f'] * pyo.value(model.W_waste_pre[s, i]) for i in model.I_processed) for s in model.S)
    total_waste_cost = waste_cost_post + waste_cost_pre
    opportunity_loss_cost = sum(scenarios[s]['prob'] * sum(params['o'][i] * pyo.value(model.UnmetDemand[s, i]) for i in model.I) for s in model.S)
    total_cost = pyo.value(model.economic_cost)

    spoilage_f = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[s, 'f']) for s in model.S)
    spoilage_post_a = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[s, 'a']) for s in model.S)
    spoilage_pre_a = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_pre[s, 'a']) for s in model.S)
    processing_loss_a = pyo.value(model.P_actual['a']) * (1 - params['e']['a'])
    spoilage_post_b = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[s, 'b']) for s in model.S)
    spoilage_pre_b = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_pre[s, 'b']) for s in model.S)
    processing_loss_b = pyo.value(model.P_actual['b']) * (1 - params['e']['b'])
    total_spoilage = spoilage_f + spoilage_post_a + spoilage_pre_a + spoilage_post_b + spoilage_pre_b
    total_processing_loss = processing_loss_a + processing_loss_b
    total_food_loss = total_spoilage + total_processing_loss

    env_scores = {}
    for level in ['low', 'middle', 'high']:
        waste_impact = sum(scenarios[s]['prob'] * (
            sum(waste_emission_factors[level][i] * pyo.value(model.W_waste_post[s, i]) for i in model.I) +
            sum(waste_emission_factors[level]['f'] * pyo.value(model.W_waste_pre[s, i]) for i in model.I_processed)
        ) for s in model.S)
        process_impact = sum(process_emission_factors[level][i] * pyo.value(model.P_actual[i]) for i in model.I_processed)
        env_scores[level] = {'total': waste_impact + process_impact, 'waste': waste_impact, 'process': process_impact}

    print(f"期待総コスト: {total_cost:,.3f} 円\n")
    print("--- 総コストの内訳 ---")
    print(f"  期待保管コスト        : {total_holding_cost:,.3f}")
    print(f"  加工コスト            : {processing_cost:,.3f}")
    print(f"  期待廃棄コスト        : {total_waste_cost:,.3f} (加工後 {waste_cost_post:,.3f}, 加工前 {waste_cost_pre:,.3f})")
    print(f"  期待機会損失コスト    : {opportunity_loss_cost:,.3f}")
    print("-----------------------------")
    print(f"  内訳の合計            : {(total_holding_cost + processing_cost + total_waste_cost + opportunity_loss_cost):,.3f}\n")
    print("--- 食品ロス（廃棄量）の内訳 ---")
    print("【生鮮品】")
    print(f"  - 劣化ロス (f)   : {spoilage_f:.3f} kg\n")
    print("【加工品】")
    print("  - 製品 A:")
    print(f"    - 劣化ロス   : {(spoilage_post_a + spoilage_pre_a):.3f} kg (加工後 {spoilage_post_a:.3f}, 加工前 {spoilage_pre_a:.3f})")
    print(f"    - 加工ロス   : {processing_loss_a:.3f} kg")
    print("  - 製品 B:")
    print(f"    - 劣化ロス   : {(spoilage_post_b + spoilage_pre_b):.3f} kg (加工後 {spoilage_post_b:.3f}, 加工前 {spoilage_pre_b:.3f})")
    print(f"    - 加工ロス   : {processing_loss_b:.3f} kg")
    print("-----------------------------")
    print(f"総食品ロス量（劣化+加工）: {total_food_loss:.3f} kg")
    print(f"    内訳 - 劣化ロス : {total_spoilage:.3f} kg")
    print(f"         - 加工ロス : {total_processing_loss:.3f} kg\n")
    print("--- 環境負荷の相対的評価 ---")
    for level, scores in env_scores.items():
        print(f"  {level.capitalize():<7} 負荷: {scores['total']:.3f} ポイント (廃棄 {scores['waste']:.3f}, 加工 {scores['process']:.3f})")

# =================================================================
# モジュール7: 詳細シミュレーション結果の可視化ダッシュボード
# =================================================================
def create_dashboard(df):
    if df.empty:
        print("データが空のため、ダッシュボードを作成できません。")
        return
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("在庫レベルの推移 (kg)", "期間ごとの生産量 (kg)",
                        "期間ごとの廃棄量 (kg)", "原料配分率 (%)"),
        specs=[[{}, {}], [{}, {'type': 'domain'}]]
    )
    inv_post_cols, inv_pre_cols = ['Inv_post_f', 'Inv_post_a', 'Inv_post_b'], ['Inv_pre_a', 'Inv_pre_b']
    colors = px.colors.qualitative.Plotly
    for i, col in enumerate(inv_post_cols):
        fig.add_trace(go.Bar(x=df['period'], y=df[col] , name=f'加工後-{col[-1]}', legendgroup='inv_post', marker_color=colors[i]), row=1, col=1)
    for i, col in enumerate(inv_pre_cols):
         fig.add_trace(go.Bar(x=df['period'], y=df[col], name=f'加工前-{col[-1]}', legendgroup='inv_pre', marker_color=colors[i+3], opacity=0.7), row=1, col=1)
    fig.update_layout(barmode='stack', bargap=0.1)
    fig.add_trace(go.Bar(x=df['period'], y=df['P_a'], name='タルト(a)生産', marker_color=colors[1]), row=1, col=2)
    fig.add_trace(go.Bar(x=df['period'], y=df['P_b'], name='シロップ(b)生産', marker_color=colors[2]), row=1, col=2)
    waste_cols = ['W_post_f', 'W_post_a', 'W_post_b', 'W_pre_a', 'W_pre_b']
    for i, col in enumerate(waste_cols):
        fig.add_trace(go.Bar(x=df['period'], y=df[col], name=col, marker_color=px.colors.qualitative.Pastel[i]), row=2, col=1)
    total_alloc = df[['X_f', 'X_a', 'X_b']].sum()
    fig.add_trace(go.Pie(labels=['生鮮(f)', 'タルト用(a)', 'シロップ用(b)'], values=total_alloc, hole=.3), row=2, col=2)
    fig.update_layout(height=800, title_text=f"詳細シミュレーションダッシュボード (λ = {df['weight'].iloc[0]} )", legend_title_text="凡例")
    fig.update_xaxes(title_text="期間")
    fig.update_yaxes(title_text="重量 (kg)", row=1, col=1)
    fig.update_yaxes(title_text="重量 (kg)", row=1, col=2)
    fig.update_yaxes(title_text="重量 (kg)", row=2, col=1)
    fig.show()

# =================================================================
# モジュール8: メイン実行ブロック
# =================================================================
if __name__ == '__main__':
        SIMULATION_PERIODS = 12

        # パレート分析で使用する共通の需要シナリオセットを事前に生成する
        print("="*80)
        print(f"{SIMULATION_PERIODS}期間分の共通需要シナリオを生成しています...")
        print("="*80)
        master_scenarios = []
        for t in range(SIMULATION_PERIODS): # 1期間分の生成*12
            scenarios_for_period_t = {
                f"s_{i+1}": {
                    'prob': 1 / num_scenarios_per_period,
                    'd': {p: max(0, np.random.normal(demand_params['mean'][p], demand_params['std'][p])) for p in products}
                } for i in range(num_scenarios_per_period)
            }
            master_scenarios.append(scenarios_for_period_t)
        print("シナリオの生成が完了しました。\n")

        # 最初の期間のシナリオを使って、単期間問題としての最大・最小を見積もる
        norm_limits = calculate_normalization_bounds(
            initial_inventory_post, initial_inventory_pre, C_supply,
            master_scenarios[0], params, capacities
        )

        # -----------------------------------------------------------------------------
        # λ（環境重視度）の範囲を 0.0 ～ 1.0 (比率) に設定
        # -----------------------------------------------------------------------------
        zoom_range_start = 0.0
        zoom_range_end = 1.0
        num_points_zoom = 21 # 0.0, 0.05, 0.1, ..., 1.0

        env_weights_linear = np.linspace(zoom_range_start, zoom_range_end, num_points_zoom)
        df_results_linear = run_pareto_analysis(env_weights_linear, simulation_periods=SIMULATION_PERIODS, master_scenarios=master_scenarios,  normalization_limits=norm_limits, title=f"詳細スキャン")

        if not df_results_linear.empty:
            print("\n--- λ（環境重視度）と各指標の関係 ---")
            fig_lambda_linear_separate = make_subplots(
                rows=2, cols=1,
                shared_xaxes=True,
                vertical_spacing=0.05,
            )

            # 総コスト
            fig_lambda_linear_separate.add_trace(go.Scatter(
                x=df_results_linear['weight'], y=df_results_linear['total_cost'],
                name='総コスト', mode='lines+markers',
                line=dict( dash='solid', width=2),
                marker=dict(symbol='circle', size=6)
            ), row=1, col=1)

            # 総環境負荷
            fig_lambda_linear_separate.add_trace(go.Scatter(
                x=df_results_linear['weight'], y=df_results_linear['total_env_score'],
                name='総環境負荷', mode='lines+markers',
                line=dict(dash='solid', width=2),
                marker=dict(symbol='circle', size=6)
            ), row=2, col=1)

            fig_lambda_linear_separate.update_layout(
                    title_text=f'λ={zoom_range_start}～{zoom_range_end}における各指標の関係',
                    height=600,
                    showlegend=False,
                    plot_bgcolor='white',
                    paper_bgcolor='white',
                    title_font=dict(size=20),
                    font_color='black'
            )
            grid_color_mono='darkgray'

              # Y軸 (1段目)
            fig_lambda_linear_separate.update_yaxes(
                  title_text="総コスト（C）",
                  title_font=dict(size=18, color='black'),
                  tickfont=dict(size=14, color='black'),
                  showgrid=True,
                  gridcolor=grid_color_mono,
                  zeroline=False,
                  linecolor='black',
                  row=1, col=1
            )
            # Y軸 (2段目)
            fig_lambda_linear_separate.update_yaxes(
                  title_text="総環境負荷スコア（E）",
                  title_font=dict(size=18, color='black'),
                  tickfont=dict(size=14, color='black'),
                  showgrid=True,
                  gridcolor=grid_color_mono,
                  zeroline=False,
                  linecolor='black',
                  row=2, col=1
            )

            # X軸 (共通)
            fig_lambda_linear_separate.update_xaxes(
                  title_text='λ (環境重視度)',
                  title_font=dict(size=18, color='black'),
                  tickfont=dict(size=14, color='black'),
                  showgrid=False,
                  gridcolor=grid_color_mono,
                  linecolor='black',
                  zeroline=False,
                  row=2, col=1 # 最後のグラフにのみX軸タイトルを表示
            )
            fig_lambda_linear_separate.show()


            # --- パレートフロンティア ---
            # (コスト vs 環境負荷)
            print("\n--- グラフ２: パレートフロンティア ---")
            fig_pareto_frontier = go.Figure()
            fig_pareto_frontier.add_trace(go.Scatter(
                x=df_results_linear['total_env_score'],
                y=df_results_linear['total_cost'],
                mode='lines+markers',
                text=df_results_linear['weight'].apply(lambda w: f'λ={w:.1f}'), # ホバーテキスト
                hovertemplate='<b>環境負荷</b>: %{x:.2f}<br><b>総コスト</b>: %{y:,.0f} 円<br><b>%{text}</b><extra></extra>',
                line=dict(color='green'),   # 線の色
                marker=dict(color='green')  # 点の色

            ))

            fig_pareto_frontier.update_layout(
                title_text='パレートフロンティア（コストと環境負荷のトレードオフ）',
                xaxis_title='総環境負荷スコア (E)',
                yaxis_title='総コスト (C)',
                height=500,
                showlegend=False,
                title_font=dict(size=20),
                plot_bgcolor='white',     # グラフ背景を白に
                paper_bgcolor='white',    # グラフ全体の背景を白に
                font_color='black',       # フォントを黒に

                # X軸のスタイル
                xaxis=dict(
                    showgrid=True,
                    gridcolor=grid_color_mono,
                    linecolor='black',
                    zeroline=False
                ),
                # Y軸のスタイル
                yaxis=dict(
                    showgrid=True,
                    gridcolor=grid_color_mono,
                    linecolor='black',
                    zeroline=False
                )
            )

            fig_pareto_frontier.show()

            print("\n" + "="*80)
            print("【比較表】")
            print("="*80)
            display_columns = ['weight', 'total_cost', 'total_env_score', 'total_objective']
            print(df_results_linear[display_columns].round(2))
            print("="*80)

        # --- ステップ3: 選択したλで詳細分析 ---
        chosen_weight = 0.5 # λ=0.5 (コストと環境を50:50で考慮)
        print("\n" + "="*80)
        print(f"ステップ3: 選択したλ={chosen_weight} での詳細分析")
        print("="*80)

        # 1期間目の詳細分析
        print("--- 1期間目の詳細分析 ---")
        first_period_scenarios = master_scenarios[0] # 共通シナリオの1期目を渡す
        _, first_period_model = solve_multi_objective_model(initial_inventory_post, initial_inventory_pre, C_supply, first_period_scenarios, params, capacities, chosen_weight)
        if first_period_model:
            analyze_and_print_period_details(first_period_model, first_period_scenarios, params)
        else:
            print("詳細分析のための最適解が見つかりませんでした。")

        # 12ヶ月間のダッシュボード表示
        print("\n--- 12ヶ月間のダッシュボード ---")
        detailed_df = run_two_stage_simulation(simulation_periods=SIMULATION_PERIODS, env_weight=chosen_weight,
                                               initial_inventory_post=initial_inventory_post,
                                               initial_inventory_pre=initial_inventory_pre,
                                               master_scenarios=master_scenarios,
                                               verbose=False)

        if not detailed_df.empty:
            print(f"総コスト: {detailed_df['cost'].sum():,.2f} 円")
            print(f"総環境負荷スコア: {detailed_df['env_score'].sum():,.2f}")
            print("\nダッシュボードを生成しています...")
            create_dashboard(detailed_df)

12期間分の共通需要シナリオを生成しています...
シナリオの生成が完了しました。

正規化のための基準値(レンジ)を計算中...
  Cost Range: 16,548 ~ 36,483
  Env  Range: 5.0 ~ 54.2
詳細スキャンを開始...

完了: 10.98 秒

--- λ（環境重視度）と各指標の関係 ---



--- グラフ２: パレートフロンティア ---



【比較表】
    weight  total_cost  total_env_score  total_objective
0     0.00   170949.04           580.05        170949.04
1     0.05   172119.79           576.98        163542.65
2     0.10   174167.51           572.81        156808.04
3     0.15   177797.93           565.14        151213.01
4     0.20   182844.84           558.17        146387.50
5     0.25   189893.44           551.01        142557.84
6     0.30   211148.01           539.09        147965.33
7     0.35   260341.17           516.88        169402.67
8     0.40   298054.18           500.90        179032.87
9     0.45   329098.81           493.97        181226.63
10    0.50   360896.58           482.05        180689.31
11    0.55   425079.80           459.03        191538.38
12    0.60   551578.10           418.57        220882.38
13    0.65   591236.41           410.69        207199.69
14    0.70   591236.41           410.69        177658.40
15    0.75   591236.41           410.69        148117.12
16    0.80   591236.41  

### コード


In [7]:


# products = ['f', 'a', 'b']
# processed_products = ['a', 'b']

# # --- 基本データ ---
# WEIGHT_TART_g = 350
# WEIGHT_SYRUP_g = 400
# WEIGHT_PER_PIECE = {'a': WEIGHT_TART_g / 1000, 'b': WEIGHT_SYRUP_g / 1000}

# # コスト関連
# COST_FREEZER_monthly = 389.17
# COST_OVEN_monthly = 155
# COST_GAS_monthly = 1048.5
# PACKING_COST_PER_PIECE = {'a': 63, 'b': 230}

# # 加工効率
# e_a = 0.8
# e_b = 0.9

# # 加工コスト計算 (円/kg-input)
# p_a = (COST_OVEN_monthly + (PACKING_COST_PER_PIECE['a'] * 80)) / ((WEIGHT_PER_PIECE['a'] * 60) / e_a)
# p_b = (COST_GAS_monthly + (PACKING_COST_PER_PIECE['b'] * 40)) / ((WEIGHT_PER_PIECE['b'] * 30) / e_b)

# # --- ★重要: 環境負荷係数の適正化 (kg-CO2e/kg) ---
# # 参考値:
# # - 食品廃棄焼却: 約 0.1(生ごみ) ~ 2.5(プラ包装含む) kg-CO2/kg
# # - 食品加工(電力/ガス): 日本の電力係数 ~0.45kg-CO2/kWh, 都市ガス ~2.2kg-CO2/m3
# # - 1個あたりの製造で数百g-CO2が出ると仮定しkg換算

# waste_emission_factors = {
#     # 生鮮品は水分多いため焼却負荷は低めだが、輸送廃棄コスト含む
#     'f': 0.2,
#     # 加工品はプラスチック包装や個包装ごとの廃棄となるため負荷が高いと仮定
#     'a': 2.5,
#     'b': 2.0
# }

# process_emission_factors = {
#     # オーブン焼成などはエネルギーを食うため高め
#     'a': 1.5,
#     # 煮沸・充填工程
#     'b': 0.8
# }

# # --- その他のパラメータ ---
# LIFESPAN_FRESH_days = 60
# LIFESPAN_TART_days = 120
# LIFESPAN_SYRUP_days = 365

# params = {
#     'h': {'f': 5, 'a': COST_FREEZER_monthly / 80, 'b': 1},
#     'p': {'f': 0, 'a': p_a, 'b': p_b},
#     'gamma': {'f': min(1.0, 30/LIFESPAN_FRESH_days), 'a': min(1.0, 30/LIFESPAN_TART_days), 'b': min(1.0, 30/LIFESPAN_SYRUP_days)},
#     'e': {'f': 1, 'a': e_a, 'b': e_b},
#     'phi': {'f': 100, 'a': 200, 'b': 200}, # 廃棄処理費用(円/kg)
#     'o': {'f': 500, 'a': 3800/WEIGHT_PER_PIECE['a'], 'b': 2200/WEIGHT_PER_PIECE['b']} # 機会損失(販売価格相当)
# }

# capacities = {
#     'U': {'f': 40, 'a': 80, 'b': 120},
#     'u': {'a': 100 * WEIGHT_PER_PIECE['a'], 'b': 50 * WEIGHT_PER_PIECE['b']} # 少し余裕を持たせる
# }

# # 供給量を需要に対して少しタイトにする（または過剰にする）ことで意思決定の余地を作る
# C_supply = 25

# demand_params = {
#     'mean': {'f': 5, 'a': 30 * WEIGHT_PER_PIECE['a'], 'b': 20 * WEIGHT_PER_PIECE['b']},
#     'std': {'f': 2, 'a': 15 * WEIGHT_PER_PIECE['a'], 'b': 10 * WEIGHT_PER_PIECE['b']}
# }

# # --- ★重要: スケーリング係数 ---
# # 環境負荷 1kg-CO2 を、最適化計算上で 何円 と同等の重みとして扱うか。
# # これにより (1-λ)*円 + λ*(CO2 * SCALE) のバランスをとる。
# # 炭素税的な考え方で、例えば 1000円/kg-CO2 くらいの重みを持たせないと、
# # 数十万円のコストに対して無視されてしまう。
# CO2_SCALE_FACTOR = 1000.0

# num_scenarios_per_period = 20

# # =================================================================
# # モジュール3: 最適化モデル
# # =================================================================
# def solve_model(initial_inv_post, initial_inv_pre, supply, scenarios, env_weight):
#     if solver is None: return None, None
#     model = pyo.ConcreteModel()
#     model.I = pyo.Set(initialize=products)
#     model.I_proc = pyo.Set(initialize=processed_products)
#     model.S = pyo.Set(initialize=scenarios.keys())

#     # 変数
#     model.X = pyo.Var(model.I, domain=pyo.NonNegativeReals)
#     model.P = pyo.Var(model.I_proc, domain=pyo.NonNegativeReals)
#     model.I_post = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals)
#     model.I_pre = pyo.Var(model.S, model.I_proc, domain=pyo.NonNegativeReals)
#     model.Sold = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals)
#     model.W_post = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals)
#     model.W_pre = pyo.Var(model.S, model.I_proc, domain=pyo.NonNegativeReals)
#     model.Lost = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals)

#     # --- 目的関数 ---
#     # 1. 経済コスト (円)
#     cost_expr = (
#         sum(scenarios[s]['prob'] * sum(params['h'][i] * model.I_post[s, i] for i in model.I) for s in model.S) +
#         sum(scenarios[s]['prob'] * sum(params['h']['f'] * model.I_pre[s, i] for i in model.I_proc) for s in model.S) +
#         sum(params['p'][i] * model.P[i] for i in model.I_proc) +
#         sum(scenarios[s]['prob'] * sum(params['phi'][i] * model.W_post[s, i] for i in model.I) for s in model.S) +
#         sum(scenarios[s]['prob'] * sum(params['phi']['f'] * model.W_pre[s, i] for i in model.I_proc) for s in model.S) +
#         sum(scenarios[s]['prob'] * sum(params['o'][i] * model.Lost[s, i] for i in model.I) for s in model.S)
#     )

#     # 2. 環境負荷 (kg-CO2)
#     env_expr = (
#         sum(scenarios[s]['prob'] * sum(waste_emission_factors[i] * model.W_post[s, i] for i in model.I) for s in model.S) +
#         sum(scenarios[s]['prob'] * sum(waste_emission_factors['f'] * model.W_pre[s, i] for i in model.I_proc) for s in model.S) +
#         sum(process_emission_factors[i] * model.P[i] for i in model.I_proc)
#     )

#     model.cost = pyo.Expression(expr=cost_expr)
#     model.env = pyo.Expression(expr=env_expr)

#     # 目的関数: スケール補正を適用
#     # 環境負荷に係数(CO2_SCALE_FACTOR)を掛けて、円換算の価値を高める
#     model.obj = pyo.Objective(
#         expr = (1 - env_weight) * model.cost + env_weight * (model.env * CO2_SCALE_FACTOR),
#         sense = pyo.minimize
#     )

#     # 制約
#     model.c = pyo.ConstraintList()
#     model.c.add(sum(model.X[i] for i in model.I) == supply) # 供給全量割り当て

#     for i in model.I_proc:
#         model.c.add(model.P[i] * params['e'][i] <= capacities['u'][i])
#         model.c.add(model.P[i] <= initial_inv_pre.get(i, 0)*(1-params['gamma']['f']) + model.X[i])

#     for s in model.S:
#         # 生鮮品バランス
#         model.c.add(initial_inv_post['f']*(1-params['gamma']['f']) + model.X['f'] == model.Sold[s,'f'] + model.I_post[s,'f'])

#         for i in model.I_proc:
#             # 加工品バランス
#             model.c.add(initial_inv_post.get(i,0)*(1-params['gamma'][i]) + model.P[i]*params['e'][i] == model.Sold[s,i] + model.I_post[s,i])
#             # 加工前在庫バランス
#             model.c.add((initial_inv_pre.get(i,0)*(1-params['gamma']['f']) + model.X[i]) - model.P[i] == model.I_pre[s,i])
#             # 加工前廃棄
#             model.c.add(model.W_pre[s,i] == initial_inv_pre.get(i,0)*params['gamma']['f'])

#         for i in model.I:
#             model.c.add(model.Sold[s,i] <= scenarios[s]['d'][i])
#             model.c.add(model.Sold[s,i] + model.Lost[s,i] == scenarios[s]['d'][i])
#             model.c.add(model.W_post[s,i] == initial_inv_post.get(i,0)*params['gamma'][i])
#             model.c.add(model.I_post[s,i] <= capacities['U'][i])

#     try:
#         res = solver.solve(model, tee=False)
#         if res.solver.termination_condition == pyo.TerminationCondition.optimal:
#             decisions = {'X':{i:pyo.value(model.X[i]) for i in model.I}, 'P':{i:pyo.value(model.P[i]) for i in model.I_proc}}
#             return decisions, model
#     except: pass
#     return None, None

# # =================================================================
# # モジュール4: シミュレーションループ
# # =================================================================
# def run_sim(periods, weight, init_post, init_pre, scenarios):
#     if solver is None: return pd.DataFrame()
#     inv_post, inv_pre = init_post.copy(), init_pre.copy()
#     history = []

#     for t in range(periods):
#         decs, model = solve_model(inv_post, inv_pre, C_supply, scenarios[t], weight)
#         if not decs: break

#         # 期待値計算
#         exp_cost = pyo.value(model.cost)
#         exp_env = pyo.value(model.env)

#         # 次期在庫更新 (簡易的にシナリオ1の結果を採用、本来は実現値サンプリング)
#         s1 = list(scenarios[t].keys())[0]

#         row = {
#             'period': t+1, 'weight': weight, 'cost': exp_cost, 'env': exp_env,
#             'total_obj': (1-weight)*exp_cost + weight*exp_env*CO2_SCALE_FACTOR
#         }
#         history.append(row)

#         # 在庫更新
#         for p in products:
#             inv_post[p] = pyo.value(model.I_post[s1, p])
#         for p in processed_products:
#             inv_pre[p] = pyo.value(model.I_pre[s1, p]) # 生鮮の劣化を考慮した残り

#     return pd.DataFrame(history)

# # =================================================================
# # メイン: パレート分析
# # =================================================================
# if __name__ == '__main__':
#     if solver:
#         # シナリオ生成
#         SIM_PERIODS = 12
#         np.random.seed(42)
#         master_scenarios = []
#         for _ in range(SIM_PERIODS):
#             scens = {f"s_{i}": {'prob': 1/20, 'd': {p: max(0, np.random.normal(demand_params['mean'][p], demand_params['std'][p])) for p in products}} for i in range(20)}
#             master_scenarios.append(scens)

#         print(f"スケーリング係数 (CO2_SCALE_FACTOR): {CO2_SCALE_FACTOR}")
#         print("分析中...")

#         results = []
#         # λの刻みを細かくする
#         for w in np.linspace(0, 1.0, 21):
#             df = run_sim(SIM_PERIODS, w, initial_inventory_post, initial_inventory_pre, master_scenarios)
#             if not df.empty:
#                 results.append(df.sum(numeric_only=True).to_dict())
#                 results[-1]['weight'] = w # sumで足されてしまうので上書き

#         res_df = pd.DataFrame(results)

# import plotly.graph_objects as go
# import pandas as pd
# import numpy as np

# # 前回の実行結果(df_results_linear)が残っている前提です
# if 'df_results_linear' in locals() and not df_results_linear.empty:

#     # -------------------------------------------------------------------------
#     # 1. U字カーブの作成 (統合コスト分析)
#     # -------------------------------------------------------------------------
#     # 仮想的な炭素価格 (Carbon Price) を設定 (円/kg-CO2)
#     # 例: 欧州ETS価格等を参考に、高め(100円)と低め(10円)で比較
#     carbon_prices = [10, 50, 100, 500]

#     fig_u_shape = go.Figure()

#     for cp in carbon_prices:
#         # 統合コスト = 経済コスト + (環境負荷 * 炭素価格)
#         # ※ここでの環境負荷は前回計算したスケーリング前の生の値に戻して計算すべきですが
#         #   今回は簡易的に total_env_score をそのまま使用します（スケーリング済みなら調整が必要）

#         # 前回のコードで CO2_SCALE_FACTOR を掛けていた場合は、それを考慮する必要があります。
#         # ここでは df_results_linear['total_env_score'] が kg-CO2 そのものだと仮定します。

#         total_integrated_cost = df_results_linear['total_cost'] + (df_results_linear['total_env_score'] * cp)

#         fig_u_shape.add_trace(go.Scatter(
#             x=df_results_linear['weight'], # X軸をλにする
#             y=total_integrated_cost,
#             mode='lines+markers',
#             name=f'炭素価格 {cp}円/kg',
#             hovertemplate='λ: %{x:.2f}<br>統合コスト: %{y:,.0f}円'
#         ))

#         # 最適点（最小値）に印をつける
#         min_idx = total_integrated_cost.idxmin()
#         min_point = total_integrated_cost.min()
#         best_weight = df_results_linear.loc[min_idx, 'weight']

#         fig_u_shape.add_trace(go.Scatter(
#             x=[best_weight], y=[min_point],
#             mode='markers',
#             marker=dict(size=12, symbol='star', color='red'),
#             name=f'最適点 (CP={cp})',
#             showlegend=False
#         ))

#     fig_u_shape.update_layout(
#         title='【U字化】環境重視度(λ)と統合コストの関係',
#         xaxis_title='環境重視度 λ (0=コスト重視, 1=環境重視)',
#         yaxis_title='統合コスト (経済コスト + 環境税相当額)',
#         height=600,
#         plot_bgcolor='white',
#         paper_bgcolor='white',
#         font_color='black'
#     )
#     fig_u_shape.update_xaxes(showgrid=True, gridcolor='lightgray')
#     fig_u_shape.update_yaxes(showgrid=True, gridcolor='lightgray')

#     fig_u_shape.show()

#     print("解説：Y軸が一番低いところが、その炭素価格設定における「最適解」です。")
#     print("炭素価格が高いほど、グラフの底（最適点）は右側（環境重視）に移動します。")


#     # -------------------------------------------------------------------------
#     # 2. ニーポイント（バランス点）の可視化
#     # -------------------------------------------------------------------------
#     # 正規化して原点(0,0)からの距離が最も近い点、あるいは(1,1)から遠い点などを探す

#     # コストと環境負荷を 0~1 に正規化
#     c_min, c_max = df_results_linear['total_cost'].min(), df_results_linear['total_cost'].max()
#     e_min, e_max = df_results_linear['total_env_score'].min(), df_results_linear['total_env_score'].max()

#     norm_cost = (df_results_linear['total_cost'] - c_min) / (c_max - c_min)
#     norm_env = (df_results_linear['total_env_score'] - e_min) / (e_max - e_min)

#     # 理想点 (コスト最小かつ環境最小 = 左下の原点) からのユークリッド距離
#     # ただしパレートフロンティアは凸なので、ここでは (0,0) に一番近い点を探す（※正規化済み空間で）
#     distances = np.sqrt(norm_cost**2 + norm_env**2)
#     knee_idx = distances.idxmin()
#     knee_weight = df_results_linear.loc[knee_idx, 'weight']

#     print("\n" + "="*50)
#     print(f"【バランス最適点（ニーポイント）】")
#     print(f"推奨される λ (weight): {knee_weight:.2f}")
#     print(f"その時のコスト: {df_results_linear.loc[knee_idx, 'total_cost']:,.0f} 円")
#     print(f"その時の環境負荷: {df_results_linear.loc[knee_idx, 'total_env_score']:.2f} kg-CO2")
#     print("="*50)

#     # グラフで確認
#     fig_knee = go.Figure()
#     fig_knee.add_trace(go.Scatter(
#         x=df_results_linear['total_env_score'],
#         y=df_results_linear['total_cost'],
#         mode='lines+markers',
#         name='パレートフロンティア'
#     ))

#     # ニーポイントを強調
#     fig_knee.add_trace(go.Scatter(
#         x=[df_results_linear.loc[knee_idx, 'total_env_score']],
#         y=[df_results_linear.loc[knee_idx, 'total_cost']],
#         mode='markers+text',
#         marker=dict(size=15, color='red', symbol='star'),
#         text=['バランス最適点'],
#         textposition="top right",
#         name='バランス最適点'
#     ))

#     fig_knee.update_layout(
#         title='パレートフロンティア上のバランス最適点',
#         xaxis_title='環境負荷 (kg-CO2)',
#         yaxis_title='総コスト (円)',
#         height=500,
#         plot_bgcolor='white'
#     )
#     fig_knee.update_xaxes(showgrid=True, gridcolor='lightgray')
#     fig_knee.update_yaxes(showgrid=True, gridcolor='lightgray')

#     fig_knee.show()

# else:
#     print("前のシミュレーション結果(df_results_linear)が見つかりません。")

スケーリング係数 (CO2_SCALE_FACTOR): 1000.0
分析中...


解説：Y軸が一番低いところが、その炭素価格設定における「最適解」です。
炭素価格が高いほど、グラフの底（最適点）は右側（環境重視）に移動します。

【バランス最適点（ニーポイント）】
推奨される λ (weight): 0.05
その時のコスト: 179,313 円
その時の環境負荷: 83.12 kg-CO2
